In [27]:
import numpy as np
import pandas as pd
import numpy.matlib
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


In [28]:
policy_data=pd.read_csv("https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv")
mobility_data=pd.read_csv("/Users/Tara/Downloads/Global_Mobility_Report (3).csv")


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
policy_data


,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,United States,USA,NaN,NaN,NAT_GOV,20200101,0.0,NaN,NaN,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
1,United States,USA,NaN,NaN,NAT_GOV,20200102,0.0,NaN,NaN,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
2,United States,USA,NaN,NaN,NAT_GOV,20200103,0.0,NaN,NaN,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
3,United States,USA,NaN,NaN,NAT_GOV,20200104,0.0,NaN,NaN,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
4,United States,USA,NaN,NaN,NAT_GOV,20200105,0.0,NaN,NaN,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13669,United States,USA,Wyoming,US_WY,STATE_ALL,20200910,2.0,0.0,NaN,1.0,...,37.04,37.04,47.62,47.62,46.15,46.15,45.45,45.45,50.0,50.0
13670,United States,USA,Wyoming,US_WY,STATE_ALL,20200911,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0
13671,United States,USA,Wyoming,US_WY,STATE_ALL,20200912,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0
13672,United States,USA,Wyoming,US_WY,STATE_ALL,20200913,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0


In [30]:
    # getting rid of the months of january, february, and september from policy data,
    no_jan_policy_data=~policy_data.Date.astype(str).str.contains("202001")
    policy_data=policy_data[no_jan_policy_data]
    
    no_feb_policy_data=~policy_data.Date.astype(str).str.contains("202002")
    policy_data=policy_data[no_feb_policy_data]
    
    no_sept_policy_data=~policy_data.Date.astype(str).str.contains("Virgin Islands")
    policy_data=policy_data[no_sept_policy_data]


In [31]:
# getting rid of US virgin islands from policy data
no_usvi_policy_data=~policy_data.RegionName.astype(str).str.contains("202009")
policy_data=policy_data[no_usvi_policy_data]


In [32]:
# further cleaning the policy data
policy_data=policy_data[["RegionName", "Jurisdiction", "Date", 'StringencyIndexForDisplay', 'GovernmentResponseIndexForDisplay', 'ContainmentHealthIndexForDisplay', 'EconomicSupportIndexForDisplay']]
policy_data=policy_data[policy_data.Jurisdiction=="STATE_ALL"]
policy_data.drop('Jurisdiction', axis=1, inplace=True)
policy_data.dropna(axis=0, how="any", inplace=True)


In [33]:
policy_data

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay
318,Alaska,20200301,11.11,10.26,12.12,0.0
319,Alaska,20200302,11.11,10.26,12.12,0.0
320,Alaska,20200303,11.11,10.26,12.12,0.0
321,Alaska,20200304,11.11,10.26,12.12,0.0
322,Alaska,20200305,11.11,10.26,12.12,0.0
...,...,...,...,...,...,...
13669,Wyoming,20200910,37.04,46.15,45.45,50.0
13670,Wyoming,20200911,37.04,46.15,45.45,50.0
13671,Wyoming,20200912,37.04,46.15,45.45,50.0
13672,Wyoming,20200913,37.04,46.15,45.45,50.0


In [34]:
mobility_data

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402995,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-02,NaN,NaN,NaN,NaN,5.0,NaN
2402996,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-03,NaN,NaN,NaN,NaN,6.0,NaN
2402997,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-04,NaN,NaN,NaN,NaN,2.0,NaN
2402998,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-07,NaN,NaN,NaN,NaN,-9.0,NaN


In [35]:
# filtering mobility data so it only includes the US
mobility_data=mobility_data[mobility_data.country_region=="United States"]

# getting rid of the months of february, and september from mobility data
no_feb_mobility_data=~mobility_data.date.astype(str).str.contains("2020-02")
mobility_data=mobility_data[no_feb_mobility_data]

no_sept_mobility_data=~mobility_data.date.astype(str).str.contains("2020-09")
mobility_data=mobility_data[no_sept_mobility_data]

mobility_data


,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
1830696,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-01,13.0,9.0,18.0,7.0,3.0,-1.0
1830697,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-02,7.0,6.0,10.0,2.0,3.0,-1.0
1830698,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-03,11.0,10.0,20.0,3.0,2.0,-1.0
1830699,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-04,8.0,6.0,17.0,2.0,3.0,-1.0
1830700,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-05,8.0,7.0,21.0,2.0,3.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378675,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-08-25,NaN,NaN,NaN,NaN,-23.0,NaN
2378676,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-08-26,NaN,NaN,NaN,NaN,-22.0,NaN
2378677,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-08-27,NaN,NaN,NaN,NaN,-16.0,NaN
2378678,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-08-28,NaN,NaN,NaN,NaN,-20.0,NaN


In [36]:
# further cleaning the mobility data
# mobility_data.dropna(axis=0, how="any", inplace=True)
mobility_data=mobility_data[["country_region", "sub_region_1", "date", 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"]]
mobility_data.dropna(axis=0, how="any", inplace=True)


<ipython-input-36-5c789d4562c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mobility_data.dropna(axis=0, how="any", inplace=True)


In [37]:
mobility_data

,country_region,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
1830903,United States,Alabama,2020-03-01,16.0,10.0,21.0,4.0,-2.0,53.0
1830904,United States,Alabama,2020-03-02,8.0,4.0,5.0,2.0,0.0,-2.0
1830905,United States,Alabama,2020-03-03,20.0,15.0,5.0,2.0,-1.0,33.0
1830906,United States,Alabama,2020-03-04,7.0,0.0,2.0,0.0,1.0,-8.0
1830907,United States,Alabama,2020-03-05,7.0,6.0,4.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
2377009,United States,Wyoming,2020-08-20,7.0,15.0,17.0,-27.0,3.0,111.0
2377010,United States,Wyoming,2020-08-21,1.0,15.0,8.0,-28.0,4.0,122.0
2377014,United States,Wyoming,2020-08-25,6.0,15.0,4.0,-26.0,4.0,109.0
2377016,United States,Wyoming,2020-08-27,5.0,15.0,2.0,-28.0,4.0,97.0


In [38]:
mobility_data_states=mobility_data.sub_region_1.unique()
mobility_data_dates=mobility_data.date.unique()
mobility_data_states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [39]:
# # condensing mobility data (putting it into a new df) so there's 1 data point per day per state,
# mobility_data_condensed=pd.DataFrame(columns=["RegionName", "Date", "Retail_and_Recreational", "Grocery_and_Pharmacy", "Transit_Stations", "Workplaces", "Residential", "Parks"])

# dict_of_rows_to_average={}
# for index, row in mobility_data.iterrows():
#     for i in mobility_data_dates:
#         for j in mobility_data_states:
#             str_lookup=str(i)+str(j)
#             if str_lookup in dict_of_rows_to_average:
#                 dict_of_rows_to_average[str_lookup]=dict_of_rows_to_average[str_lookup].append(row)
#             else:
#                 dict_of_rows_to_average[str_lookup]=pd.DataFrame(columns=["country_region", "sub_region_1", "date", 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"])
#                 dict_of_rows_to_average[str_lookup]=dict_of_rows_to_average[str_lookup].append(row)


In [40]:
mobility_data2 = mobility_data.groupby(['sub_region_1','date'])['retail_and_recreation_percent_change_from_baseline', "grocery_and_pharmacy_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"].apply(np.mean).reset_index()


<ipython-input-40-1944d576671d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mobility_data2 = mobility_data.groupby(['sub_region_1','date'])['retail_and_recreation_percent_change_from_baseline', "grocery_and_pharmacy_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"].apply(np.mean).reset_index()


In [41]:
mobility_data2

,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alabama,2020-03-01,15.400000,11.300000,19.100000,3.500000,-2.200000,43.100000
1,Alabama,2020-03-02,7.545455,4.000000,3.454545,1.727273,0.545455,-10.000000
2,Alabama,2020-03-03,17.250000,14.375000,3.000000,2.625000,-1.625000,44.625000
3,Alabama,2020-03-04,6.111111,2.888889,0.333333,0.888889,0.777778,-7.222222
4,Alabama,2020-03-05,8.000000,6.750000,5.083333,1.416667,0.500000,-2.250000
...,...,...,...,...,...,...,...,...
9363,Wyoming,2020-08-27,8.000000,18.000000,17.500000,-25.500000,3.500000,174.500000
9364,Wyoming,2020-08-28,3.500000,17.000000,19.500000,-23.500000,3.000000,196.000000
9365,Wyoming,2020-08-29,7.000000,23.000000,44.000000,-8.000000,-1.000000,283.000000
9366,Wyoming,2020-08-30,11.500000,20.000000,25.500000,-7.000000,0.000000,186.000000


In [42]:
# print(mobility_data2.to_string())

In [43]:
# mobility_data[mobility_data["sub_region_1"] == "Alabama"][mobility_data["date"] == "2020-03-01"]

In [44]:
mobility_data=mobility_data2
mobility_data

,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alabama,2020-03-01,15.400000,11.300000,19.100000,3.500000,-2.200000,43.100000
1,Alabama,2020-03-02,7.545455,4.000000,3.454545,1.727273,0.545455,-10.000000
2,Alabama,2020-03-03,17.250000,14.375000,3.000000,2.625000,-1.625000,44.625000
3,Alabama,2020-03-04,6.111111,2.888889,0.333333,0.888889,0.777778,-7.222222
4,Alabama,2020-03-05,8.000000,6.750000,5.083333,1.416667,0.500000,-2.250000
...,...,...,...,...,...,...,...,...
9363,Wyoming,2020-08-27,8.000000,18.000000,17.500000,-25.500000,3.500000,174.500000
9364,Wyoming,2020-08-28,3.500000,17.000000,19.500000,-23.500000,3.000000,196.000000
9365,Wyoming,2020-08-29,7.000000,23.000000,44.000000,-8.000000,-1.000000,283.000000
9366,Wyoming,2020-08-30,11.500000,20.000000,25.500000,-7.000000,0.000000,186.000000


In [45]:
mobility_data['sub_region_1'] = np.where(mobility_data['sub_region_1'] == 'District of Columbia', 'Washington DC', mobility_data['sub_region_1'])
mobility_data["state_date_id"] = mobility_data.apply(lambda row: row.sub_region_1 + '_' + row.date.replace('-',''), axis=1)


In [46]:
policy_data["state_date_id"] = policy_data.apply(lambda row: row.RegionName + '_' + str(row.Date), axis=1)

In [47]:
mobility_data

,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline,state_date_id
0,Alabama,2020-03-01,15.400000,11.300000,19.100000,3.500000,-2.200000,43.100000,Alabama_20200301
1,Alabama,2020-03-02,7.545455,4.000000,3.454545,1.727273,0.545455,-10.000000,Alabama_20200302
2,Alabama,2020-03-03,17.250000,14.375000,3.000000,2.625000,-1.625000,44.625000,Alabama_20200303
3,Alabama,2020-03-04,6.111111,2.888889,0.333333,0.888889,0.777778,-7.222222,Alabama_20200304
4,Alabama,2020-03-05,8.000000,6.750000,5.083333,1.416667,0.500000,-2.250000,Alabama_20200305
...,...,...,...,...,...,...,...,...,...
9363,Wyoming,2020-08-27,8.000000,18.000000,17.500000,-25.500000,3.500000,174.500000,Wyoming_20200827
9364,Wyoming,2020-08-28,3.500000,17.000000,19.500000,-23.500000,3.000000,196.000000,Wyoming_20200828
9365,Wyoming,2020-08-29,7.000000,23.000000,44.000000,-8.000000,-1.000000,283.000000,Wyoming_20200829
9366,Wyoming,2020-08-30,11.500000,20.000000,25.500000,-7.000000,0.000000,186.000000,Wyoming_20200830


In [48]:
policy_data

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,state_date_id
318,Alaska,20200301,11.11,10.26,12.12,0.0,Alaska_20200301
319,Alaska,20200302,11.11,10.26,12.12,0.0,Alaska_20200302
320,Alaska,20200303,11.11,10.26,12.12,0.0,Alaska_20200303
321,Alaska,20200304,11.11,10.26,12.12,0.0,Alaska_20200304
322,Alaska,20200305,11.11,10.26,12.12,0.0,Alaska_20200305
...,...,...,...,...,...,...,...
13669,Wyoming,20200910,37.04,46.15,45.45,50.0,Wyoming_20200910
13670,Wyoming,20200911,37.04,46.15,45.45,50.0,Wyoming_20200911
13671,Wyoming,20200912,37.04,46.15,45.45,50.0,Wyoming_20200912
13672,Wyoming,20200913,37.04,46.15,45.45,50.0,Wyoming_20200913


In [49]:
inner_join_df=pd.merge(policy_data[["state_date_id", "RegionName", "Date", "StringencyIndexForDisplay", "GovernmentResponseIndexForDisplay", "ContainmentHealthIndexForDisplay", "EconomicSupportIndexForDisplay"]], mobility_data[["state_date_id", "retail_and_recreation_percent_change_from_baseline","grocery_and_pharmacy_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline" ]], on='state_date_id', how='inner')
inner_join_df


,state_date_id,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alaska_20200301,Alaska,20200301,11.11,10.26,12.12,0.0,19.0,14.5,18.0,5.5,-1.0,11.5
1,Alaska_20200302,Alaska,20200302,11.11,10.26,12.12,0.0,6.0,6.5,6.5,0.5,0.0,6.5
2,Alaska_20200303,Alaska,20200303,11.11,10.26,12.12,0.0,11.0,10.0,4.0,3.0,0.0,-5.0
3,Alaska_20200304,Alaska,20200304,11.11,10.26,12.12,0.0,10.0,8.5,7.5,3.0,-0.5,3.0
4,Alaska_20200305,Alaska,20200305,11.11,10.26,12.12,0.0,8.5,7.5,3.0,2.5,-0.5,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,Wyoming_20200827,Wyoming,20200827,37.04,46.15,45.45,50.0,8.0,18.0,17.5,-25.5,3.5,174.5
9342,Wyoming_20200828,Wyoming,20200828,37.04,46.15,45.45,50.0,3.5,17.0,19.5,-23.5,3.0,196.0
9343,Wyoming_20200829,Wyoming,20200829,37.04,46.15,45.45,50.0,7.0,23.0,44.0,-8.0,-1.0,283.0
9344,Wyoming_20200830,Wyoming,20200830,37.04,46.15,45.45,50.0,11.5,20.0,25.5,-7.0,0.0,186.0


In [50]:
inner_join_df.drop(["state_date_id"], axis=1,inplace=True)

In [51]:
inner_join_df

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alaska,20200301,11.11,10.26,12.12,0.0,19.0,14.5,18.0,5.5,-1.0,11.5
1,Alaska,20200302,11.11,10.26,12.12,0.0,6.0,6.5,6.5,0.5,0.0,6.5
2,Alaska,20200303,11.11,10.26,12.12,0.0,11.0,10.0,4.0,3.0,0.0,-5.0
3,Alaska,20200304,11.11,10.26,12.12,0.0,10.0,8.5,7.5,3.0,-0.5,3.0
4,Alaska,20200305,11.11,10.26,12.12,0.0,8.5,7.5,3.0,2.5,-0.5,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9341,Wyoming,20200827,37.04,46.15,45.45,50.0,8.0,18.0,17.5,-25.5,3.5,174.5
9342,Wyoming,20200828,37.04,46.15,45.45,50.0,3.5,17.0,19.5,-23.5,3.0,196.0
9343,Wyoming,20200829,37.04,46.15,45.45,50.0,7.0,23.0,44.0,-8.0,-1.0,283.0
9344,Wyoming,20200830,37.04,46.15,45.45,50.0,11.5,20.0,25.5,-7.0,0.0,186.0


In [52]:
inner_join_df.to_csv('/Users/Tara/Downloads/policy_mobility_df.csv', header=True)